# Analyzing borrowers’ risk of defaulting

This project is to prepare a report for a bank’s loan division. Need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
# Loading all the libraries
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
ps = PorterStemmer()
wordnet_lemma = WordNetLemmatizer()
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
df = pd.read_csv('credit_scoring_eng.csv')

## Task 1. Data exploration

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


There are minuses in [days_employed] , different sizes of letters in [education] , different names of one purpose (e.g. 'to have a wedding' and 'having a wedding'. Maybe need add [purpose_id] and [income_type_id] columns.

In [5]:
for column in df.columns:
    print('{}:'.format(column), pd.api.types.infer_dtype(df[column]))

children: integer
days_employed: floating
dob_years: integer
education: string
education_id: integer
family_status: string
family_status_id: integer
gender: string
income_type: string
debt: integer
total_income: floating
purpose: string


In [6]:
df.describe(include = 'all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


In [7]:
df.duplicated().sum()

54

In [8]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
4182,1,NaN,34,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
7808,0,NaN,57,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding
8583,0,NaN,58,bachelor's degree,0,unmarried,4,F,retiree,0,NaN,supplementary education
9238,2,NaN,34,secondary education,1,married,0,F,employee,0,NaN,buying property for renting out
9528,0,NaN,66,secondary education,1,widow / widower,2,F,retiree,0,NaN,transactions with my real estate
9627,0,NaN,56,secondary education,1,married,0,F,retiree,0,NaN,transactions with my real estate
10462,0,NaN,62,secondary education,1,married,0,F,retiree,0,NaN,buy commercial real estate


There are missing values in columns [days_employed] and [total_income]. [days_employed] need to be int64. [total_income] too, but maybe this is not necessary. No mixed types. 54 explicit duplicates.

In [9]:
print(df.isna().sum())
print()
display(df[df['days_employed'].isna()])
display(df[df['total_income'].isna()])
df[df['days_employed'].isna() & df['total_income'].isna()]

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Intermediate conclusion**

The number of rows in the filtered table match the number of missing values. Thats mean that if NaN in [days_employed] so NaN in [total_income] and vice versa.

Percentage of the missing values is about 10% compared to the whole dataset. It is a considerably large piece of data. Maybe we can fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.

I'll try to find patterns that exist regarding missing values. And perhaps I can fill in the missing data in connection with the patterns identified.

In [10]:
filtered_data = df[df['days_employed'].isna()]
filtered_data[filtered_data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
mega_filtered_data = filtered_data[filtered_data['total_income'].isna() & filtered_data['education_id'] == 1]
display(mega_filtered_data)
df[df['education_id'] == 1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21426,0,NaN,49,secondary education,1,married,0,F,employee,1,NaN,property
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [12]:
# Checking distribution
print(df['education_id'].value_counts(normalize=True))
print()
print(filtered_data['education_id'].value_counts(normalize=True))
print()
print(df['income_type'].value_counts(normalize=True))
print()
print(filtered_data['income_type'].value_counts(normalize=True))
print()
print(df['dob_years'].value_counts(normalize=True).head(5))
print()
print(filtered_data['dob_years'].value_counts(normalize=True).head(5))
print()
print(df['children'].value_counts())
print()
print(filtered_data['children'].value_counts())

1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64

1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
Name: dob_years, dtype: float64

34    0.031739
40    0.030359
42    0.029899
31    0.029899
35    0.029439
Name: dob_years, dtype: float64

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-

Unfortunately I don't see patterns in missing values.

**Conclusions**

The distribution in the original dataset similar to the distribution of the filtered table. My conclusion about missing values is that they are accidental. So we can fill in columns [days_employed] and [total_income] with mean of each group of people.
There are several groups of people, for each of them need to count mean and fill in columns [days_employed] and [total_income] with missing values.

Also I need to address other types of issues: duplicates, different registers, incorrect artifacts (e.g. minuses in [days_employed] , different sizes of letters in [education] , different names of one purpose)

## Data transformation

Let's go through each column to see what issues we may have in them.

Begin with removing duplicates and fixing educational information if required.

In [13]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df.education.unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [14]:
# Fix the registers if required
df['education'] = df['education'].str.lower()


In [15]:
# Checking all the values in the column to make sure we fixed them
df['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Check the data the `children` column

In [16]:
# Let's see the distribution of values in the `children` column
print(df['children'].unique())
print()
print(df['children'].value_counts())
print()
print(df['children'].value_counts(normalize=True))

[ 1  0  3  2 -1  4 20  5]

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


There  are strange things in the column - count of children can't be negative and 20. I think it's a data entry error. "-1" is a random addition of a minus, which means you can change this data to just "1". "20" can be either an input error "2" or an input error "0". Since the number of children is one of the important metrics we'll be testing hypotheses against, I think it's best to remove that data.

In [17]:
df['children'] = df['children'].replace(-1, 1)

df = df[df.children != 20]

print(df['children'].unique())

[1 0 3 2 4 5]


Checking the `children` column again to make sure it's all fixed

In [18]:
print(df['children'].unique())
print()
print(df['children'].value_counts())
print()
print(df['children'].value_counts(normalize=True))


[1 0 3 2 4 5]

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

0    0.659658
1    0.226817
2    0.095809
3    0.015385
4    0.001912
5    0.000420
Name: children, dtype: float64


Checking the data in the `days_employed` column

In [19]:
print(df['days_employed'].describe())
print()
df['days_employed'] = df['days_employed'].abs()
print(df['days_employed'].describe())

count     19284.000000
mean      63141.233527
std      140910.281638
min      -18388.949901
25%       -2747.876441
50%       -1204.164714
75%        -290.206221
max      401755.400475
Name: days_employed, dtype: float64

count     19284.000000
mean      67009.548818
std      139112.246407
min          24.141633
25%         927.984311
50%        2196.507938
75%        5551.712316
max      401755.400475
Name: days_employed, dtype: float64


In [20]:
notna_df = df[df['days_employed'].notna()]
print(notna_df[notna_df['days_employed'] > 19000]['dob_years'].count())
print()
notna_df[notna_df['days_employed'] > 15000].sort_values(['days_employed']).head(20)

3438



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5581,0,15079.216069,55,secondary education,1,married,0,F,civil servant,0,28601.820,housing transactions
16868,0,15193.032201,57,secondary education,1,married,0,F,civil servant,0,31199.021,building a property
19788,0,15267.541183,56,secondary education,1,married,0,M,employee,0,19676.219,car purchase
15675,0,15410.040779,65,bachelor's degree,0,married,0,F,employee,0,30208.011,buy residential real estate
7731,0,15618.063786,64,secondary education,1,married,0,F,business,0,47444.057,university education
4321,0,15773.061335,61,secondary education,1,civil partnership,1,F,employee,0,32938.974,wedding ceremony
1539,0,15785.678893,59,bachelor's degree,0,unmarried,4,F,employee,0,19130.216,transactions with commercial real estate
3974,0,15835.725775,64,secondary education,1,civil partnership,1,F,business,0,15497.365,to have a wedding
16825,0,16119.687737,64,secondary education,1,married,0,F,employee,0,14644.430,buy residential real estate
17838,0,16264.699501,59,secondary education,1,married,0,F,employee,0,8198.235,to buy a car


Amount of problematic data is high, it could've been due to some technical issues. 18000 days is 50 years. You can see in the table that 3438 rows have a value greater than 328728 in the ['days_employed'] column. Most likely the decimal point has shifted and 328728 is 3287.28 days. I will divide the table into 2 parts - where the data is before 328728 and after. In the second table, I'll move the decimal point 2 to the left (divide by 100). If the main metrics are the same, then I can join the tables.

In [21]:
notna_df_first = notna_df[notna_df['days_employed'] < 20000]
notna_df_second = notna_df[notna_df['days_employed'] > 20000]
notna_df_second['days_employed'] = notna_df_second['days_employed']/100

display(notna_df_first.describe())
print()
notna_df_second.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000
mean,0.557302,2353.798816,39.820712,0.798183,0.970844,0.087151,27837.656833
std,0.788905,2304.164544,10.663574,0.555223,1.442810,0.282066,16993.690534
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3418.824000
25%,0.000000,757.151250,32.000000,0.000000,0.000000,0.000000,17320.770500
50%,0.000000,1631.100855,39.000000,1.000000,0.000000,0.000000,24183.572500
75%,1.000000,3157.881359,48.000000,1.000000,1.000000,0.000000,33814.785500
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3438.000000,3438.000000,3438.000000,3438.000000,3438.000000,3438.000000,3438.000000
mean,0.091914,3650.127526,59.124200,0.914485,0.984875,0.052938,21943.583773
std,0.327640,210.841037,7.583268,0.517615,1.316754,0.223942,12848.298935
min,0.000000,3287.287206,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,3466.443800,56.000000,1.000000,0.000000,0.000000,13260.622500
50%,0.000000,3652.493463,60.000000,1.000000,0.000000,0.000000,18945.005500
75%,0.000000,3832.757508,64.000000,1.000000,2.000000,0.000000,27162.396750
max,4.000000,4017.554005,74.000000,4.000000,4.000000,1.000000,117616.523000


As we can see, the average [days_employed] in the first table is 2353 with an average age of 39 years. In the second table, the average [days_employed] is 3650, with an average age of 59 years. Which is generally logical - the more years, the more work experience. The last step is to understand how much the indicators will change if we connect these 2 tables. (on copy)

In [22]:
df_copy = df
notna_df_copy = df_copy[df_copy['days_employed'].notna()]
for i in notna_df_copy['days_employed']:
    if i > 19000:
        i = i/100

display(notna_df_copy.describe())
notna_df_first.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19284.000000,19284.000000,19284.000000,19284.000000,19284.000000,19284.000000,19284.000000
mean,0.474331,67009.548818,43.262186,0.818917,0.973346,0.081052,26786.846670
std,0.749848,139112.246407,12.580911,0.550497,1.421133,0.272921,16486.681895
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.984311,33.000000,1.000000,0.000000,0.000000,16482.580250
50%,0.000000,2196.507938,42.000000,1.000000,0.000000,0.000000,23203.328000
75%,1.000000,5551.712316,53.000000,1.000000,1.000000,0.000000,32539.207750
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000
mean,0.557302,2353.798816,39.820712,0.798183,0.970844,0.087151,27837.656833
std,0.788905,2304.164544,10.663574,0.555223,1.442810,0.282066,16993.690534
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3418.824000
25%,0.000000,757.151250,32.000000,0.000000,0.000000,0.000000,17320.770500
50%,0.000000,1631.100855,39.000000,1.000000,0.000000,0.000000,24183.572500
75%,1.000000,3157.881359,48.000000,1.000000,1.000000,0.000000,33814.785500
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000


Is itn't safe to remove more than 10% of data. Moreover, we do not even use this column in further analysis. So is better to ignore these values.

Let's now look at the client's age and whether there are any issues there.

In [23]:
df['dob_years'].sort_values(ascending=False).value_counts(normalize=True)


35    0.028673
40    0.028206
41    0.028206
34    0.027973
38    0.027833
42    0.027694
33    0.026994
39    0.026668
31    0.026015
36    0.025782
44    0.025409
29    0.025316
30    0.025036
48    0.025036
37    0.024850
43    0.023824
50    0.023824
32    0.023684
49    0.023544
28    0.023451
45    0.023031
27    0.022892
52    0.022519
56    0.022472
47    0.022379
54    0.022285
46    0.022006
58    0.021493
57    0.021400
53    0.021353
51    0.020840
55    0.020607
59    0.020607
26    0.018975
60    0.017530
25    0.016598
61    0.016504
62    0.016364
63    0.012541
64    0.012308
24    0.012262
23    0.011795
65    0.009045
22    0.008532
66    0.008532
67    0.007786
21    0.005128
0     0.004662
68    0.004616
69    0.003916
70    0.003030
71    0.002704
20    0.002378
72    0.001539
19    0.000653
73    0.000373
74    0.000280
75    0.000047
Name: dob_years, dtype: float64

There is 0 years. But only 0,4%. If we enter an average value here, then it will affect some age (age 39 will become the most numerous). However, our hypotheses are not related to age, so in order not to lose the data, we can either leave 0 or still change it to 39.

In [24]:
df[df['dob_years']==0].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
1149,0,934.654854,0,secondary education,1,married,0,F,business,0,32296.389,buy real estate
1175,0,370879.508002,0,secondary education,1,married,0,F,retiree,0,50231.975,to get a supplementary education
1386,0,5043.219890,0,bachelor's degree,0,married,0,M,civil servant,0,38483.779,purchase of a car
1890,0,NaN,0,bachelor's degree,0,unmarried,4,F,employee,0,NaN,housing
1898,0,370144.537021,0,secondary education,1,widow / widower,2,F,retiree,0,20384.043,to buy a car


In [25]:
df[df['dob_years']==0]['gender'].value_counts()

F    71
M    29
Name: gender, dtype: int64

Age 0 is indicated in women 2 times more often than in men. Perhaps age 0 is not a mistake, but the person simply did not want to indicate his age. We cannot fill in these missing data. Since we'll be using this column for categorization later on, I suggest we just keep it, and keep in mind that the group with age 0 is the group of people who didn't specify their age.

Now let's check the `family_status` column.

In [26]:
df['family_status'].unique()


array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

No problematic values

Now let's check the `gender` column.

In [27]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [28]:
df['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

In [29]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


Just 1 person with wrong value in `gender` column. I will delete this row.

In [30]:
df = df[df['gender'] != 'XNA']

In [31]:
df['gender'].value_counts()


F    14189
M     7259
Name: gender, dtype: int64

Now let's check the `income_type` column.

In [32]:
df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

No problematic values

Now let's see if we have any duplicates in our data.

In [33]:
df.duplicated().sum()


71

In [34]:
df = df.drop_duplicates()
df.duplicated().sum()

0

Checking the size of the dataset that we now have after first manipulations with it

In [35]:
df.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21377.000000,19283.000000,21377.000000,21377,21377.000000,21377,21377.000000,21377,21377,21377.000000,19283.000000,21377
unique,NaN,NaN,NaN,5,NaN,5,NaN,2,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,15112,NaN,12290,NaN,14127,11041,NaN,NaN,790
mean,0.475184,67012.901561,43.277307,NaN,0.816953,NaN,0.974459,NaN,NaN,0.081068,26786.543918,NaN
std,0.751965,139115.074530,12.572157,NaN,0.549000,NaN,1.421802,NaN,NaN,0.272947,16487.055797,NaN
min,0.000000,24.141633,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,927.974414,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16482.158500,NaN
50%,0.000000,2195.695527,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,5553.019016,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32536.052000,NaN


I fixed the incorrect values in the table, removed the duplicates, and prepared the table to deal with the missing values.


# Working with missing values

### Restoring missing values in `total_income`

Missing values are in columns `days_employed` and `total_income`. I will fill them with average values, relative to `education`, `dob_years` and `income_type`.

In [36]:
def assign_age_group(age):
    if age < 0 or pd.isna(age):
        return 'NA'
    elif age < 10:
        return '0 - without specify'
    elif age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    elif age < 70:
        return '60-69'
    return '70+'    

In [37]:
# Testing if the function works
assign_age_group(19)


'10-19'

In [38]:
# Creating new column based on function
df['age_group'] = df['dob_years'].apply(assign_age_group)
df['age_group']

0        40-49
1        30-39
2        30-39
3        30-39
4        50-59
         ...  
21520    40-49
21521    60-69
21522    30-39
21523    30-39
21524    40-49
Name: age_group, Length: 21377, dtype: object

In [39]:
# Checking how many values in the new column
df['age_group'].value_counts()


30-39                  5640
40-49                  5331
50-59                  4641
20-29                  3156
60-69                  2326
70+                     169
0 - without specify     100
10-19                    14
Name: age_group, dtype: int64

Creating a table that only has data without missing values. This data will be used to restore the missing values.

In [40]:
df_without_nan = df[df['days_employed'].notna()]
df_without_nan

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


In [41]:
# Looking at the mean values for income based on your identified factors
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='20-29')&(df_without_nan['income_type']=='employee')]['total_income'].mean())
print(df_without_nan[(df_without_nan['education_id']==2)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='employee')]['total_income'].mean())
print(df_without_nan[(df_without_nan['education_id']==3)&(df_without_nan['age_group']=='40-49')&(df_without_nan['income_type']=='business')]['total_income'].mean())
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='business')]['total_income'].mean())

22607.955577720208
31785.952666666668
25941.41585714286
29221.560627906976


In [42]:
# Looking at the median values for income based on your identified factors
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='20-29')&(df_without_nan['income_type']=='employee')]['total_income'].median())
print(df_without_nan[(df_without_nan['education_id']==2)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='employee')]['total_income'].median())
print(df_without_nan[(df_without_nan['education_id']==3)&(df_without_nan['age_group']=='40-49')&(df_without_nan['income_type']=='business')]['total_income'].median())
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='business')]['total_income'].median())

20192.619
26913.823
27929.098
26243.519


In [43]:
df_without_nan[(df_without_nan['education_id']==3)&(df_without_nan['age_group']=='40-49')&(df_without_nan['income_type']=='business')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 2473 to 19534
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          7 non-null      int64  
 1   days_employed     7 non-null      float64
 2   dob_years         7 non-null      int64  
 3   education         7 non-null      object 
 4   education_id      7 non-null      int64  
 5   family_status     7 non-null      object 
 6   family_status_id  7 non-null      int64  
 7   gender            7 non-null      object 
 8   income_type       7 non-null      object 
 9   debt              7 non-null      int64  
 10  total_income      7 non-null      float64
 11  purpose           7 non-null      object 
 12  age_group         7 non-null      object 
dtypes: float64(2), int64(5), object(6)
memory usage: 784.0+ bytes


As expected, because of the extremes, it is better to calculate the income by the median.
There are exceptions - but only because of a small sample (only 7 lines).


In [44]:
print(df['education_id'].unique())
print(df['age_group'].unique())
print(df['income_type'].unique())

[0 1 2 3 4]
['40-49' '30-39' '50-59' '20-29' '60-69' '0 - without specify' '70+'
 '10-19']
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


Filling in missing values

In [45]:
df['total_income'] = df['total_income'].fillna(df.groupby(['income_type','education_id','age_group'])['total_income'].transform('median'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     19283 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21374 non-null  float64
 11  purpose           21377 non-null  object 
 12  age_group         21377 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


In [46]:
df[(df['total_income'].isna())&(df['income_type'] != 'retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,NaN,transactions with commercial real estate,70+
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59
8142,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,NaN,to have a wedding,60-69


NaN remained only in the retierd column and 3 separate row, which are most likely simply the only ones in the group.
Lets check it

In [47]:
display(df[(df['education_id']==3)&(df['age_group']=='70+')&(df['income_type']=='employee')])
display(df[(df['education_id']==0)&(df['age_group']=='50-59')&(df['income_type']=='entrepreneur')])
display(df[(df['education_id']==3)&(df['age_group']=='60-69')&(df['income_type']=='civil servant')])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,NaN,transactions with commercial real estate,70+


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
8142,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,NaN,to have a wedding,60-69


We can fill these 3 rows with the median of the larger groups they belong to.

In [48]:
print(df[(df['education_id']==3)&(df['income_type']=='employee')]['total_income'].median())
print()
print(df[(df['education_id']==0)&(df['income_type']=='entrepreneur')]['total_income'].median())
print()
print(df[(df['education_id']==3)&(df['income_type']=='civil servant')]['total_income'].median())

19887.8015

79866.103

21150.696


In [49]:
df.loc[1303,'total_income'] = 19887.8015
print(df.loc[1303]['total_income'])
df.loc[5936,'total_income'] = 79866.103
print(df.loc[5936]['total_income'])
df.loc[8142,'total_income'] = 21150.696
print(df.loc[8142]['total_income'])

19887.8015
79866.103
21150.696


In [50]:
df[(df['total_income'].isna())&(df['income_type'] != 'retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     19283 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21377 non-null  float64
 11  purpose           21377 non-null  object 
 12  age_group         21377 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.8+ MB


Now in column `total_income` NaN just where `income_type` == `retiree`

###  Restoring values in `days_employed`

In [52]:
# Distribution of `days_employed` medians based on your identified parameters
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='20-29')&(df_without_nan['income_type']=='employee')]['days_employed'].mean())
print(df_without_nan[(df_without_nan['education_id']==2)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='employee')]['days_employed'].mean())
print(df_without_nan[(df_without_nan['education_id']==3)&(df_without_nan['age_group']=='40-49')&(df_without_nan['income_type']=='business')]['days_employed'].mean())
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='business')]['days_employed'].mean())
print()
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='20-29')&(df_without_nan['income_type']=='employee')]['days_employed'].median())
print(df_without_nan[(df_without_nan['education_id']==2)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='employee')]['days_employed'].median())
print(df_without_nan[(df_without_nan['education_id']==3)&(df_without_nan['age_group']=='40-49')&(df_without_nan['income_type']=='business')]['days_employed'].median())
print(df_without_nan[(df_without_nan['education_id']==1)&(df_without_nan['age_group']=='30-39')&(df_without_nan['income_type']=='business')]['days_employed'].median())

1212.5386916102145
1545.9793657043049
2074.8715172954558
1950.4195588140833

1016.8350413323262
1238.2166843108976
1151.634459637238
1646.9456145928634


I will use medians again, cause there are some extremums in top

In [53]:
df['days_employed'] = df['days_employed'].fillna(df.groupby(['income_type','education_id','age_group'])['days_employed'].transform('median'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     21374 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21377 non-null  float64
 11  purpose           21377 non-null  object 
 12  age_group         21377 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.8+ MB


In [54]:
df[(df['days_employed'].isna())&(df['income_type'] != 'retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,19887.8015,transactions with commercial real estate,70+
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,79866.1030,buy residential real estate,50-59
8142,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,21150.6960,to have a wedding,60-69


In [55]:
print(df[(df['education_id']==3)&(df['income_type']=='employee')]['days_employed'].median())
print()
print(df[(df['education_id']==0)&(df['income_type']=='entrepreneur')]['days_employed'].median())
print()
print(df[(df['education_id']==3)&(df['income_type']=='civil servant')]['days_employed'].median())


1164.6529878789477

520.8480834953765

2531.6011904356965


In [56]:
df.loc[1303,'days_employed'] = 1164.6529878789477
print(df.loc[1303]['days_employed'])
df.loc[5936,'days_employed'] = 520.8480834953765
print(df.loc[5936]['days_employed'])
df.loc[8142,'days_employed'] = 2531.6011904356965
print(df.loc[8142]['days_employed'])


1164.6529878789477
520.8480834953765
2531.6011904356965


In [57]:
df[(df['days_employed'].isna())&(df['income_type'] != 'retiree')]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     21377 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21377 non-null  float64
 11  purpose           21377 non-null  object 
 12  age_group         21377 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.8+ MB


And again in column `days_employed` NaN just where `income_type` == `retiree'`
I think no need to delete rows where `income_type` == `retiree`, cause we can analyse it separatly

## Categorization of data


In [59]:
# Printing the values for selected data for categorization
df.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [60]:
lemmas_list_all = []
for purpose in df.purpose.unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas = [l.lower() for l in lemmas]
    for i in lemmas:
        if i==',':
            continue
        else:
            lemmas_list_all.append(i)
lemmas_list_all

['purchase',
 'of',
 'the',
 'house',
 'car',
 'purchase',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'housing',
 'transaction',
 'education',
 'having',
 'a',
 'wedding',
 'purchase',
 'of',
 'the',
 'house',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'buy',
 'commercial',
 'real',
 'estate',
 'buy',
 'residential',
 'real',
 'estate',
 'construction',
 'of',
 'own',
 'property',
 'property',
 'building',
 'a',
 'property',
 'buying',
 'a',
 'second-hand',
 'car',
 'buying',
 'my',
 'own',
 'car',
 'transaction',
 'with',
 'commercial',
 'real',
 'estate',
 'building',
 'a',
 'real',
 'estate',
 'housing',
 'transaction',
 'with',
 'my',
 'real',
 'estate',
 'car',
 'to',
 'become',
 'educated',
 'second-hand',
 'car',
 'purchase',
 'getting',
 'an',
 'education',
 'car',
 'wedding',
 'ceremony',
 'to',
 'get',
 'a',
 'supplementary',
 'education',
 'purchase',
 'of',
 'my',
 'own',
 'house',
 'real',
 'estate',
 'transaction',
 'getting',
 'higher'

In [61]:
from collections import Counter
Counter(lemmas_list_all)

Counter({'purchase': 6,
         'of': 5,
         'the': 2,
         'house': 3,
         'car': 9,
         'supplementary': 2,
         'education': 7,
         'to': 6,
         'have': 1,
         'a': 9,
         'wedding': 3,
         'housing': 3,
         'transaction': 4,
         'having': 1,
         'for': 2,
         'my': 4,
         'family': 1,
         'buy': 4,
         'real': 7,
         'estate': 7,
         'commercial': 2,
         'residential': 1,
         'construction': 1,
         'own': 4,
         'property': 4,
         'building': 2,
         'buying': 3,
         'second-hand': 2,
         'with': 2,
         'become': 1,
         'educated': 1,
         'getting': 2,
         'an': 1,
         'ceremony': 1,
         'get': 1,
         'higher': 1,
         'profile': 1,
         'university': 2,
         'renting': 1,
         'out': 1,
         'renovation': 1,
         'going': 1})

In [62]:
housing_catogory=['house','estate','property','housing']
cars_catogory=['car']
education_catogory=['education','educated','university']
wedding_catogory=['wedding']

In [63]:
def lemmatization_func(line):
    words = nltk.word_tokenize(line)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas = [l.lower() for l in lemmas]
    if any(word in lemmas for word in housing_catogory):
        return 'housing'
    elif any(word in lemmas for word in cars_catogory):
        return 'car'
    elif any(word in lemmas for word in education_catogory):
        return 'education'
    elif any(word in lemmas for word in wedding_catogory):
        return 'wedding'
    else:
        return 'other'

In [64]:
df['purpose_categ']=df['purpose'].apply(lemmatization_func)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_categ
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,housing
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,housing
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49,housing
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,car
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39,housing
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,car


In [65]:
df['purpose_categ'].value_counts()

housing      10774
car           4290
education     3998
wedding       2315
Name: purpose_categ, dtype: int64

In [66]:
df['total_income'].describe()

count     21377.000000
mean      26477.045510
std       15745.110699
min        3306.762000
25%       17188.946000
50%       22928.480000
75%       31655.315000
max      362496.645000
Name: total_income, dtype: float64

In [67]:
def avg_income_func(income):
    if income >= 32751.753500:
        return 'fourth quartile'
    elif income >= 24041.342000:
        return 'third quartile'
    elif income >= 17973.487000:
        return 'second quartile'
    else:
        return 'first quartile'

df['income_quartile'] = df['total_income'].apply(avg_income_func)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_categ,income_quartile
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,housing,fourth quartile
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car,first quartile
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,housing,second quartile
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education,fourth quartile
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding,third quartile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49,housing,fourth quartile
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,car,third quartile
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39,housing,first quartile
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,car,fourth quartile


In [68]:
print(df['education'].value_counts())
edu_dict = df[['education_id','education']]
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)
edu_dict.sort_values('education_id')

secondary education    15112
bachelor's degree       5236
some college             741
primary education        282
graduate degree            6
Name: education, dtype: int64


,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [69]:
print(df['family_status'].value_counts())
fam_dict = df[['family_status_id','family_status']]
fam_dict = fam_dict.drop_duplicates().reset_index(drop=True)
fam_dict.sort_values('family_status_id')

married              12290
civil partnership     4138
unmarried             2801
divorced              1193
widow / widower        955
Name: family_status, dtype: int64


,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


In [70]:
df.purpose_categ.value_counts()

housing      10774
car           4290
education     3998
wedding       2315
Name: purpose_categ, dtype: int64

Now we have 4 categories for `purpose` column and `total_income` by quartile

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [71]:
# Checking the children data and paying back on time
df.pivot_table(values='debt',index='children',aggfunc = ['mean','count'])

# Calculating default-rate based on the number of children



,mean,count
,debt,debt
children,,
0,0.075444,14090
1,0.091658,4855
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


**Conclusion**

The largest default rate is 4 children. But the sample is small, only 41 lines. Parents of 3 children have one of the smallest indicator, but the sample is also small. However, we can conclude that people without children can be given an increased credit rating, because with a large sample, they have the smallest default rate.

**Is there a correlation between family status and paying back on time?**

In [72]:
# Check the family status data and paying back on time
df.pivot_table(values='debt',index='family_status',aggfunc = ['mean','count'])

,mean,count
,debt,debt
family_status,,
civil partnership,0.093040,4138
divorced,0.070411,1193
married,0.075509,12290
unmarried,0.097465,2801
widow / widower,0.065969,955


**Conclusion**

Married and divorced are the groups with the lowest default rating. Widow / widower has an even lower chance of default, but the group itself is statistically small. Unmarried and civil partnership are the least reliable groups. ALso widowed people are, as a rule, older, and they are more responsible. And most likely they already have adult children who can help with payments. In general, we can say this: the most irresponsible are those who have never been married

**Is there a correlation between income level and paying back on time?**

In [73]:
# Checking the income level data and paying back on time
debt_by_inc_stat = pd.pivot_table(df,
                                 values='debt',
                                 index='income_quartile',
                                 aggfunc = 'mean')
debt_by_inc_stat

,debt
income_quartile,
first quartile,0.079915
fourth quartile,0.068780
second quartile,0.091216
third quartile,0.083464


**Conclusion**

You can clearly see that the more income, the less chance of default. The second quartile has the highest value. However, the values of the first three quartiles do not differ much. But for people in the fourth quartile, we can improve their credit rating.

**How does credit purpose affect the default rate?**

In [74]:
# Check the percentages for default rate for each credit purpose and analyze them
debt_by_purp_stat = pd.pivot_table(df,
                                 values='debt',
                                 index='purpose_categ',
                                 aggfunc = 'mean')
debt_by_purp_stat

,debt
purpose_categ,
car,0.093473
education,0.092296
housing,0.072397
wedding,0.079050


**Conclusion**

People who take out a loan for real estate have the least chance of default. And those who take a loan for study and a car have a higher chance of default - we can lower their credit rating.


# General Conclusion 

From this analysis, we can draw some conclusions and try to rank people before issuing a loan. And, accordingly, to issue a loan with a higher percentage to those people who are lower in the rating. For example, people with an income of more than 32751 (fourth quartile) will have a high rating, perhaps some kind of multiplying factor. And unmarried people taking out a car loan will have a low rating.

That is, the analysis will help in the production of this assessment system.


In [75]:
df.groupby(['purpose_categ'])['debt'].mean().reset_index().sort_values(by='debt')

,purpose_categ,debt
2,housing,0.072397
3,wedding,0.079050
1,education,0.092296
0,car,0.093473


Percentage of non-return on time for clients who take out a loan for housing 7,2397%

Percentage of non-return on time for clients who take out a loan for car 9,3473%